In [1]:
import pandas as pd
from datetime import date

import datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from prophet import Prophet

Загрузка тренировочного датасета. В процессе разработки все подготовительные работы с датасетом сделаны в другом ноутбуке, здесь мы используем подготовленный датасет

In [2]:
df = pd.read_csv('sp_sales_task\\train.csv')

In [3]:
df.head()

,Unnamed: 0,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_sales_in_rub,month_year,price_units,price_pred,...,year,month,week,day,holiday,liquidity,top_st_revenue,cluster_kmean,cluster_st,st_id_pr_sku_id
0,0,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2022-10-20,1,5,825,2022-10,165.0,0.006738,...,2022,10,3,20,0,1,0,241.0,5,c81e728d9d4c2f636f067f89cc14862cc7b711619071c9...
1,1,42a0e188f5033bc65bf8d78622277c4e,68f441429e73dbd33b81f95da31576e9,2023-01-29,0,4,196,2023-01,49.0,0.018316,...,2023,1,6,29,1,1,0,3.0,0,42a0e188f5033bc65bf8d78622277c4e68f441429e73db...
2,2,1ecfb463472ec9115b10c292ef8bc986,22988026fe85fde60d8bf777ab6bc3b4,2023-03-02,0,1,78,2023-03,78.0,0.367879,...,2023,3,3,2,0,1,0,385.0,0,1ecfb463472ec9115b10c292ef8bc98622988026fe85fd...
3,3,16a5cdae362b8d27a1d8f8c7b78b4330,e6fc24ca846339d204b0b50e9f772484,2023-02-17,0,1,161,2023-02,161.0,0.367879,...,2023,2,4,17,0,1,0,11.0,0,16a5cdae362b8d27a1d8f8c7b78b4330e6fc24ca846339...
4,4,42a0e188f5033bc65bf8d78622277c4e,3474f0e025c7deb7e127a35763bc5512,2022-11-03,0,3,123,2022-11,41.0,0.049787,...,2022,11,3,3,0,1,0,762.0,1,42a0e188f5033bc65bf8d78622277c4e3474f0e025c7de...


In [4]:
df = df.rename(columns = {'pr_sales_in_rub' : 'y'})
df['date'] = pd.to_datetime(df['date'])
df = df.drop(columns = ['pr_sales_in_units',  'price_units', 'price_pred', 'price_units_log',  'month_year'], axis = 1)

Разделяем данные по времени на тренировочные и тестовые. 

In [5]:
date_lag = 15
predictions_period = df['date'].max() - timedelta(date_lag)
train = df.loc[df['date'] < predictions_period]
test = df.loc[df['date'] >= predictions_period]

In [6]:
# Проверяем, что разделение прошло успешно
print("Даты тренировочного датасета: с", train['date'].min(), "по:", train['date'].max())
print("Даты тренировочного датасета: с", test['date'].min(), "по:", test['date'].max())

Даты тренировочного датасета: с 2022-08-01 00:00:00 по: 2023-07-02 00:00:00
Даты тренировочного датасета: с 2023-07-03 00:00:00 по: 2023-07-18 00:00:00


In [7]:
# Создим признаки и целевые признаки.
X_train = train.drop(['y', 'date'], axis=1)
y_train = train['y']

In [8]:
# Перебор очень долгий. В связи с тем, что мы пока тестируем модели, используем дефолтные параметры.

# %%time

# # Попробуем изменить гиперпараметры. Будем использовать кросс-валидацию с использованием 
# # библиотеки TimeSeriesSplit, что исключит подглядывание 
# model_cbr_tune = CatBoostRegressor(loss_function='RMSE')
# tscv = TimeSeriesSplit(n_splits=5)
# parameters_cbr_tune = {'iterations': [1000],
#         'learning_rate': [0.03],#, 0.1],
#         'depth': [6],#, 8],
#         'l2_leaf_reg': [1]#, 3]
#               }
# model_cbr_tune = GridSearchCV(model_cbr_tune, 
#                         parameters_cbr_tune, 
#                         cv = tscv, 
#                         n_jobs=-1,
#                               scoring = 'neg_mean_squared_error',
#                              verbose=True).fit(X_train, y_train, 
#                                                verbose=False, plot=True)


Для корректного создания резолютирующего файла предсказания делаем по всем комбинациям магазин-товар. 

In [9]:
cat_features = ['st_id', 'pr_sku_id', 'st_id_pr_sku_id', 'pr_group_id', 'pr_cat_id', 'pr_subcat_id', 'st_city_id', 'st_division_code']

In [10]:
model_cbr_tune = CatBoostRegressor(cat_features=cat_features, loss_function='RMSE').fit(X_train, y_train, 
                                               verbose=False, plot=True)


model_cbr_tune.get_feature_importance(prettified=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

,Feature Id,Importances
0,cluster_st,20.664861
1,pr_sku_id,14.704158
2,month,12.757840
3,pr_cat_id,8.284719
4,day,7.562962
5,pr_subcat_id,6.853532
6,st_id_pr_sku_id,6.228596
7,week,6.196363
8,pr_sales_type_id,5.123938
9,cluster_kmean,3.550440


In [11]:
# Определяем параметры перебора по связкам магазин-товар.
a = set(test['st_id'])
b = set(test['pr_sku_id'])

In [12]:
%%time
sales_submission_test = pd.DataFrame(columns=['st_id', 'pr_sku_id', 'date']) #Для вычисления метрики
for i in a:
    for j in b:
        test_pred = test[(test['st_id'] == i) & (test['pr_sku_id'] == j)] # Выделяем часть датасета в разрезе нужной связки
       
        df_1 = pd.DataFrame(columns=['st_id', 'pr_sku_id', 'date', 'y']) # Резолютирующий датасет по предсказаниям

        df_1['st_id'], df_1['pr_sku_id'], df_1['date'], df_1['y'] = \
            test_pred['st_id'], test_pred['pr_sku_id'], test_pred['date'], test_pred['y']
       
        X_test = test_pred.drop(['y', 'date'], axis=1) # Выделяем данные для предсказаний

        df_1['target'] = model_cbr_tune.predict(X_test) # Предсказание

        sales_submission_test = pd.concat([sales_submission_test, df_1], ignore_index=False)

<timed exec>:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


CPU times: total: 44 s
Wall time: 1min 8s


In [13]:
# sales_submission_test.groupby('date').count()

In [14]:
# Вычисляем метрику
wape = 100 * (sales_submission_test['y'] - sales_submission_test['target']).abs().sum() / sales_submission_test['y'].sum()
wape

56.54808241848806

In [15]:
# Посмотрим, что получилось
sales_submission_test.reset_index(drop= True , inplace= True )
sales_submission_test

,st_id,pr_sku_id,date,y,target
0,53e19f3dbb211f20b20b45668303c1b6,57eaf5659b0371dbef7bb6cee4a14e71,2023-07-11,110.0,74.377868
1,53e19f3dbb211f20b20b45668303c1b6,57eaf5659b0371dbef7bb6cee4a14e71,2023-07-05,147.0,70.307135
2,53e19f3dbb211f20b20b45668303c1b6,57eaf5659b0371dbef7bb6cee4a14e71,2023-07-10,80.0,55.502647
3,53e19f3dbb211f20b20b45668303c1b6,57eaf5659b0371dbef7bb6cee4a14e71,2023-07-18,37.0,66.181387
4,53e19f3dbb211f20b20b45668303c1b6,57eaf5659b0371dbef7bb6cee4a14e71,2023-07-18,99.0,29.057919
...,...,...,...,...,...
38631,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7000b6d0d0ba91f81a7562a22bcc4f5c,2023-07-11,65.0,79.933791
38632,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7000b6d0d0ba91f81a7562a22bcc4f5c,2023-07-07,65.0,144.098613
38633,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7000b6d0d0ba91f81a7562a22bcc4f5c,2023-07-08,124.0,190.164191
38634,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7000b6d0d0ba91f81a7562a22bcc4f5c,2023-07-16,131.0,218.282634


In [16]:
# Создаем нужный файл в соответствии с заданием и записываем его

sales_submission = sales_submission_test.drop(['y'], axis=1)
sales_submission.to_csv('sales_submission.csv')

In [17]:
# sales_submission_test['wape'] = 100 * (sales_submission_test['y'] - sales_submission_test['target']).abs() / sales_submission_test['y']
# sales_submission_test[sales_submission_test['wape'] < 10]